In [1]:
from scipy import sparse as sp
import json
import pickle
import sys
import numpy as np
import implicit
import threadpoolctl
from tqdm import tqdm

/home/VoidWalker/Code/cs307/Bee-roll/machine_learning/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Limiting the threads
threadpoolctl.threadpool_limits(1, "blas")

In [3]:
sys.path.append("../src")
from config import NUM_PARTS, PATH_TO_ALS_MODEL
from models import ALS
from utils import MovieEncoder, make_coo_row, get_part_path

In [4]:
file = open("../models/movie.encoder", "rb")
encoder = pickle.load(file)
file.close()

In [5]:
model = implicit.als.AlternatingLeastSquares(factors=64)

In [6]:
rows = []
users = []
empty_rows = []
for index in range(NUM_PARTS):
    for data in tqdm((json.loads(part) for part in open(get_part_path(index)))):
        users.append(data["user_id"])
        empty_rows.append(make_coo_row([], encoder))
        rows.append(make_coo_row(data["review_history"], encoder))
train_mat = sp.vstack(rows)
empty_mat = sp.vstack(empty_rows)
users = np.array(users)

33300it [00:04, 7392.39it/s]
33045it [00:03, 8462.65it/s]
33011it [00:03, 8522.31it/s]
33117it [00:03, 8610.00it/s]
33299it [00:03, 8894.41it/s]
33052it [00:03, 8445.24it/s]
32895it [00:03, 8425.62it/s]
33145it [00:03, 8765.42it/s]
32881it [00:04, 8175.02it/s]
33230it [00:03, 8749.57it/s]


In [7]:
model.fit(train_mat)

/home/VoidWalker/Code/cs307/Bee-roll/machine_learning/env/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.971874475479126 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:41<00:00,  2.80s/it]


In [8]:
model.partial_fit_users(users, empty_mat) # Removing info about the users

/home/VoidWalker/Code/cs307/Bee-roll/machine_learning/env/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0002741813659667969 seconds
  warnings.warn(


In [9]:
trained_model = ALS(model)

In [10]:
file = open(PATH_TO_ALS_MODEL, 'wb')
pickle.dump(trained_model, file)
file.close()

In [5]:
file = open(PATH_TO_ALS_MODEL, 'rb')
trained_model = pickle.load(file)
file.close()